In [46]:
# import pandas as pden 
from pathlib import Path
from tqdm import tqdm
import modin as pd

In [47]:
# input
data_dir = Path("/Users/oholm/annad/ordtidni/Gigaword/output/")
input_file_path = data_dir.glob('*/**/*')
filepaths = [item for item in input_file_path if (item.is_file() and str(item).endswith('.parquet'))]
# output
output_filepath = data_dir/'word_occurrence_counts.parquet'

In [ ]:
%%time
print('about to process', len(filepaths), 'files')
word_occurance_count = pd.DataFrame({'occurance_count':[0]}, index=['barn']) # cold start dataframe
# display(word_occurance_count)
print('----'*8, 'start iteration over files', '----'*8)
for filepath in tqdm(filepaths):
#     print(filepath.parent)
    corpus_doc = pd.read_parquet(filepath)
#     print('read in dataframe')
#     display(corpus_doc.head(2))
    noun_mask = corpus_doc['POS'].str.startswith('n')
#     print('there are', sum(noun_mask), 'nouns in the set')
    nouns_only_df = corpus_doc[noun_mask]
#     print('nouns only')
#     display(nouns_only_df.sample(3))
#     all_nouns_list = nouns_only_df['Lemma'].unique()
    noun_occurrence_count =\
        nouns_only_df[['Lemma', 'POS']]\
        .groupby(by='Lemma')\
        .count()\
        ['POS']\
        .rename('additions')\
        
    word_occurance_count = pd.concat([word_occurance_count, noun_occurrence_count], axis=1).fillna(0)
    word_occurance_count['occurance_count'] = word_occurance_count['occurance_count'] + word_occurance_count['additions']
    word_occurance_count.drop('additions', axis=1,inplace=True)

#     print('there are', len(word_occurance_count), 'noun lemmas')

output_filepath = data_dir/'word_occurrence_counts.parquet'
word_occurance_count.to_parquet(output_filepath)

  0%|          | 0/661 [00:00<?, ?it/s]

about to process 661 files
-------------------------------- start iteration over files --------------------------------


 90%|████████▉ | 593/661 [40:37<13:28, 11.90s/it]

In [19]:
word_occurance_count.shape

(28420, 1)

In [ ]:
top10_nouns_freq = word_occurance_count.sort_values(by='occurance_count', ascending=False).head(60).reset_index()
top10_nouns_freq

In [27]:
print('there are', sum(word_occurance_count['occurance_count'] < 2.0), 'single occurence lemmas in corpus'

0